In [1]:
# --- Run this cell FIRST (Colab / normal Jupyter) ---
!pip install folium pandas

import folium
import pandas as pd

# Folium plugins
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon


In [5]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon

In [9]:
# Make sure the CSV file is in the same folder or uploaded in Colab
df = pd.read_csv("/content/dataset_part_2.csv")
df.head()


,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class
0,1,2010-06-04,Falcon 9,6104.959412,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857,0
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857,0
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857,0
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093,0
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857,0


In [11]:
# Get one row per launch site (to get unique coordinates)
sites_df = df.groupby('LaunchSite', as_index=False).first()

# Center of all sites
center_lat = sites_df['Latitude'].mean()
center_long = sites_df['Longitude'].mean()

# Create base map
site_map = folium.Map(location=[center_lat, center_long], zoom_start=4)

# Add markers for each launch site
for _, row in sites_df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['LaunchSite'],
        icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
    ).add_to(site_map)

site_map

In [13]:
# Create a new map (or you can add to the old one)
launch_map = folium.Map(location=[center_lat, center_long], zoom_start=4)

# Use MarkerCluster so many markers don’t clutter the view
marker_cluster = MarkerCluster().add_to(launch_map)

for _, row in df.iterrows():
    # Choose marker color based on 'Class'
    if row['Class'] == 1:
        color = 'green'     # Success
        outcome = 'Success'
    else:
        color = 'red'       # Failure
        outcome = 'Failure'

    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=4,
        popup=f"{row['LaunchSite']} - {outcome}",
        color=color,
        fill=True,
        fill_color=color
    ).add_to(marker_cluster)

launch_map

In [14]:
from math import radians, sin, cos, asin, sqrt

def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great-circle distance between two points on Earth (km).
    """
    # Convert to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    R = 6371  # Radius of Earth in km
    return R * c


In [17]:
# Pick a launch site to analyze (change name if needed)
site_name = "KSC LC 39A" # Corrected from "KSC LC-39A"

site_row = sites_df[sites_df['LaunchSite'] == site_name].iloc[0]
site_lat, site_long = site_row['Latitude'], site_row['Longitude']

# Example nearby points (replace with correct ones from your lab)
points = {
    "Nearest City":      (28.3922, -80.6077),   # Example coord
    "Coastline":         (28.5,    -80.5),      # Example coord
    "Railway":           (28.45,   -80.65),     # Example coord
    "Highway":           (28.47,   -80.62)      # Example coord
}

# Create map centered on the chosen launch site
distance_map = folium.Map(location=[site_lat, site_long], zoom_start=9)

# Add launch site marker
folium.Marker(
    location=[site_lat, site_long],
    popup=site_name,
    icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
).add_to(distance_map)

# Draw lines and show distance labels
for name, (lat, lon) in points.items():
    # Add marker for point
    folium.Marker(
        location=[lat, lon],
        popup=name,
        icon=folium.Icon(color='gray')
    ).add_to(distance_map)

    # Draw line
    folium.PolyLine(
        locations=[[site_lat, site_long], [lat, lon]],
        weight=2
    ).add_to(distance_map)

    # Calculate distance
    d_km = haversine(site_lat, site_long, lat, lon)

    # Put distance text near the middle of the line
    mid_lat = (site_lat + lat) / 2
    mid_lon = (site_long + lon) / 2

    folium.map.Marker(
        [mid_lat, mid_lon],
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 10pt; color: black">{name}: {d_km:.2f} km</div>',
        )
    ).add_to(distance_map)

distance_map